# Approximation Algorithms - Assignment 2
### Group 2: Christoph Kern, Johannes Gabriel Sindlinger

Install the hyperloglog library and read data files

In [2]:
!curl https://raw.githubusercontent.com/rasmus-pagh/apx/main/data/words_danish.txt -o words_danish.txt
!curl https://raw.githubusercontent.com/rasmus-pagh/apx/main/data/words_english.txt -o words_english.txt

import hyperloglog
import copy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  258k  100  258k    0     0   790k      0 --:--:-- --:--:-- --:--:--  789k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4135k  100 4135k    0     0  4982k      0 --:--:-- --:--:-- --:--:-- 4982k


The code below reads two lists of words from files and prints statistics using a hyperloglog sketch

In [4]:
def create_hll(filename, relative_error):
  hll = hyperloglog.HyperLogLog(relative_error)
  with open(filename, 'r') as f:
    for word in f:
      hll.add(word)
  return hll

relative_error = 0.01
hll_danish = create_hll('words_danish.txt', relative_error)
hll_english = create_hll('words_english.txt', relative_error)

print(f'HLL reports about {len(hll_danish)} Danish words and {len(hll_english)} English words')

hll_combined = copy.deepcopy(hll_english) # Create a copy of the HLL
hll_combined.update(hll_danish) # Merge the other HLL into the combined one
print(f'Combined the two languages have about {len(hll_combined)} words')

HLL reports about 25642 Danish words and 372669 English words
Combined the two languages have about 395672 words


## Task:

Your task is to add code, or modify the code, to compute information about the number of distinct substrings in the two languages. For example, the word "pop" has 6 distinct substrings, namely "" (the empty string), "p", "o", "po", "op" and "pop". Note that "pp" is not a substring since the letters are not consecutive.


1. Use a hyperloglog data structure with relative error 0.01 to compute upper and lower bounds on the number of distinct substrings in Danish and English, respectively. You may assume that HLL is guaranteed to return a number with the stated relative error.

In [8]:
def create_hll_substrings(filename, relative_error):
  hll = hyperloglog.HyperLogLog(relative_error)
  with open(filename, 'r') as f:
    for word in f:
        n = len(word)
        hll.add("")
        for i in range(n):
            for j in range(i + 1, n + 1):
                hll.add(word[i:j])
  return hll


def print_statistics(estimate, absolute_error, text):
    print(f"{estimate} {text} [{estimate - absolute_error}, {estimate + absolute_error}]")


relative_error = 0.01

hll_danish_substrings = create_hll_substrings('words_danish.txt', relative_error)
danish_substrings_estimate = len(hll_danish_substrings)
danish_substrings_absolute_error = relative_error * danish_substrings_estimate
print_statistics(danish_substrings_estimate, danish_substrings_absolute_error, "Danish substrings")

hll_english_substrings = create_hll_substrings('words_english.txt', relative_error)
english_substrings_estimate = len(hll_english_substrings)
english_substrings_absolute_error = relative_error * english_substrings_estimate
print_statistics(english_substrings_estimate, english_substrings_absolute_error, "English substrings")

414464 Danish substrings [410319.36, 418608.64]
3928867 English substrings [3889578.33, 3968155.67]


2. Based on the answer from 1), and a cardinality estimate for the union, give upper and lower bounds on the number of common substrings in the two languages.

In [9]:
hll_combined_substrings = copy.deepcopy(hll_english_substrings)
hll_combined_substrings.update(hll_danish_substrings)
combined_substrings_estimate = len(hll_combined_substrings)
combined_substrings_absolute_error = relative_error * combined_substrings_estimate
print_statistics(combined_substrings_estimate, combined_substrings_absolute_error, "combined substrings")

common_substrings_estimate = english_substrings_estimate + danish_substrings_estimate - combined_substrings_estimate
common_substrings_absolute_error = english_substrings_absolute_error + danish_substrings_absolute_error + combined_substrings_absolute_error
print_statistics(common_substrings_estimate, common_substrings_absolute_error, "common substrings")

4292008 combined substrings [4249087.92, 4334928.08]
51323 common substrings [-35030.39, 137676.39]


3. Suppose that the intersection size is at least 1% of each of the two sets. How small relative error on the cardinality estimates would you need to estimate the number of common substrings up to a relative error of 10%? (NB! The implementation provided does not support very small relative errors.)